In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['MAC']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('membrane attack complex', 469),
 ('minimum alveolar concentration', 223),
 ('mycobacterium avium complex', 162),
 ('mitral annular calcification', 45),
 ('macronucleus', 27),
 ('minimum alveolar anesthetic concentration', 27),
 ('medium access control', 25),
 ('myeloablative conditioning', 24),
 ('macrophages', 23),
 ('minimal alveolar concentration', 22),
 ('mitochondrial apoptosis induced channel', 17),
 ('macitentan', 16),
 ('medial arterial calcification', 16),
 ('mucinous adenocarcinoma', 16),
 ('m avium complex', 15),
 ('mycobacterium avium intracellulare complex', 15),
 ('monitored anesthesia care', 11),
 ('mitral annulus calcification', 11),
 ('mitral annular calcium', 10),
 ('mos4 associated complex', 10),
 ('monolithic activation carbon', 8),
 ('methacrylamide chitosan', 8),
 ('minimum anesthetic concentration', 8),
 ('methylascochlorin', 7),
 ('membrane attack complex of complement', 7),
 ('multiple antigen construct', 6),
 ('metabolic acidosis', 4),
 ('mouse artificial c

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-10-10 01:07:17] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'m avium complex': 'MESH:D015269',
  'macitentan': 'CHEBI:CHEBI:76607',
  'macronucleus': 'GO:GO:0031039',
  'macrophages': 'MESH:D008264',
  'maintained': 'ungrounded',
  'mammary adenocarcinoma': 'DOID:DOID:3458',
  'maternal': 'ungrounded',
  'mean activation count': 'ungrounded',
  'media access control': 'ungrounded',
  'medial arterial calcification': 'HP:HP:0012456',
  'medium access control': 'ungrounded',
  'melaleuca alternifolia concentration': 'ungrounded',
  'membrane attack complex': 'FPLX:MAC',
  'membrane attack complex of complement': 'FPLX:MAC',
  'metabolic acidosis': 'DOID:DOID:0050758',
  'methacrylamide chitosan': 'ungrounded',
  'methylascochlorin': 'PUBCHEM:6441595',
  'minimal alveolar concentration': 'NCIT:C139130',
  'minimum alveolar anaesthetic concentration': 'NCIT:C139130',
  'minimum alveolar anesthetic concentration': 'NCIT:C139130',
  'minimum alveolar concentration': 'NCIT:C139130',
  'minimum anesthetic concentration': 'ungrounded',
  'mitochondria

In [20]:
grounding_map, names, pos_labels = [{'m avium complex': 'MESH:D015269',
  'macitentan': 'CHEBI:CHEBI:76607',
  'macronucleus': 'GO:GO:0031039',
  'macrophages': 'MESH:D008264',
  'maintained': 'ungrounded',
  'mammary adenocarcinoma': 'DOID:DOID:3458',
  'maternal': 'ungrounded',
  'mean activation count': 'ungrounded',
  'media access control': 'ungrounded',
  'medial arterial calcification': 'HP:HP:0012456',
  'medium access control': 'ungrounded',
  'melaleuca alternifolia concentration': 'ungrounded',
  'membrane attack complex': 'FPLX:MAC',
  'membrane attack complex of complement': 'FPLX:MAC',
  'metabolic acidosis': 'DOID:DOID:0050758',
  'methacrylamide chitosan': 'ungrounded',
  'methylascochlorin': 'PUBCHEM:6441595',
  'minimal alveolar concentration': 'NCIT:C139130',
  'minimum alveolar anaesthetic concentration': 'NCIT:C139130',
  'minimum alveolar anesthetic concentration': 'NCIT:C139130',
  'minimum alveolar concentration': 'NCIT:C139130',
  'minimum anesthetic concentration': 'ungrounded',
  'mitochondrial apoptosis induced channel': 'mitochondrial',
  'mitral annular calcification': 'HP:HP:0005136',
  'mitral annular calcium': 'HP:HP:0004382',
  'mitral annulus calcification': 'HP:HP:0004382',
  'monitored anesthesia care': 'ungrounded',
  'monkey adapting components': 'ungrounded',
  'monocarbonyl analogs of curcumin': 'ungrounded',
  'monolithic activation carbon': 'ungrounded',
  'mos4 associated complex': 'ungrounded',
  'mouse artificial chromosome': 'MESH:D022201',
  'mucinous adenocarcinoma': 'MESH:D002288',
  'multiple antigen construct': 'ungrounded',
  'musa acuminata cavendish banana': 'ungrounded',
  'mycobacterium avium complex': 'MESH:D015269',
  'mycobacterium avium intracellulare complex': 'MESH:D015269',
  'myeloablative conditioning': 'NCIT:C131679'},
 {'MESH:D015269': 'Mycobacterium avium Complex',
  'CHEBI:CHEBI:76607': 'macitentan',
  'GO:GO:0031039': 'macronucleus',
  'MESH:D008264': 'Macrophages',
  'DOID:DOID:3458': 'breast adenocarcinoma',
  'HP:HP:0012456': 'Medial arterial calcification',
  'FPLX:MAC': 'MAC',
  'DOID:DOID:0050758': 'metabolic acidosis',
  'PUBCHEM:6441595': '4-O-Methylascochlorin',
  'NCIT:C139130': 'Minimum Alveolar Concentration 50 Percent',
  'mitochondrial': 'mitochondrial_apoptosis_induced_channel',
  'HP:HP:0005136': 'Mitral annular calcification',
  'HP:HP:0004382': 'Mitral valve calcification',
  'MESH:D022201': 'Chromosomes, Artificial',
  'MESH:D002288': 'Adenocarcinoma, Mucinous',
  'NCIT:C131679': 'Myeloablative Conditioning'},
 ['FPLX:MAC', 'HP:HP:0005136', 'MESH:D015269', 'NCIT:C139130', 'GO:GO:0031039',
 'NCIT:C131679', 'MESH:D008264', 'MESH:D002288', 'HP:HP:0012456', 'PUBCHEM:6441595']]

In [10]:
excluded_longforms = []

In [21]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [12]:
additional_entities = {}

In [13]:
unambiguous_agent_texts = {}

In [22]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [15]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [16]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [17]:
intersection1

[('HGNC:17981', 'HGNC:17981', 29)]

In [18]:
intersection2

[('ungrounded', 'HGNC:17981', 0)]

In [20]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=['RTCA', 'RTCD1', 'RPC', 'RTC1', 'RTC'])
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [ ]:
names.update(additional_entitie)

In [23]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-10 01:50:00] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-10 01:50:21] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.9475911234792523 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [16]:
classifier.stats

{'label_distribution': {'NCIT:C139130': 243,
  'FPLX:MAC': 360,
  'MESH:D008264': 15,
  'MESH:D015269': 150,
  'ungrounded': 60,
  'DOID:DOID:3458': 3,
  'PUBCHEM:6441595': 7,
  'DOID:DOID:0050758': 4,
  'mitochondrial': 14,
  'HP:HP:0004382': 17,
  'HP:HP:0005136': 25,
  'GO:GO:0031039': 21,
  'MESH:D002288': 10,
  'NCIT:C131679': 19,
  'HP:HP:0012456': 10,
  'CHEBI:CHEBI:76607': 9,
  'MESH:D022201': 3},
 'f1': {'mean': 0.953616, 'std': 0.01069},
 'precision': {'mean': 0.929589, 'std': 0.016737},
 'recall': {'mean': 0.980736, 'std': 0.009054},
 'ungrounded': {'f1': {'mean': 0.724569, 'std': 0.143599},
  'pr': {'mean': 0.616667, 'std': 0.187083},
  'rc': {'mean': 0.921111, 'std': 0.075064}},
 'PUBCHEM:6441595': {'f1': {'mean': 0.866667, 'std': 0.163299},
  'pr': {'mean': 0.8, 'std': 0.244949},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'NCIT:C139130': {'f1': {'mean': 0.962853, 'std': 0.02006},
  'pr': {'mean': 0.995833, 'std': 0.008333},
  'rc': {'mean': 0.933254, 'std': 0.043308}},
 'DOID:D

In [24]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [27]:
disamb.dump(model_name, results_path)

In [25]:
print(disamb.info())

Disambiguation model for MAC

Produces the disambiguations:
	4-O-Methylascochlorin*	PUBCHEM:6441595
	Adenocarcinoma, Mucinous*	MESH:D002288
	Chromosomes, Artificial	MESH:D022201
	MAC*	FPLX:MAC
	Macrophages*	MESH:D008264
	Medial arterial calcification*	HP:HP:0012456
	Minimum Alveolar Concentration 50 Percent*	NCIT:C139130
	Mitral annular calcification*	HP:HP:0005136
	Mitral valve calcification	HP:HP:0004382
	Mycobacterium avium Complex*	MESH:D015269
	Myeloablative Conditioning*	NCIT:C131679
	breast adenocarcinoma	DOID:DOID:3458
	macitentan	CHEBI:CHEBI:76607
	macronucleus*	GO:GO:0031039
	metabolic acidosis	DOID:DOID:0050758
	mitochondrial_apoptosis_induced_channel	mitochondrial

Class level metrics:
--------------------
Grounding                                	Count	F1     
                                      MAC*	360	0.97034
Minimum Alveolar Concentration 50 Percent*	243	0.96285
              Mycobacterium avium Complex*	150	  0.951
                               Ungrounded	 60	0.724

In [26]:
model_to_s3(disamb)